## Productivity Report

#### For Picking, Packing, QC, ConsoRacking, ConsoPicking

In [1]:
#Import Libraries
import psycopg2
import pandas as pd
import numpy as np
import time

In [2]:
#Establish Connecttion
conn = psycopg2.connect(
    host="host",
    database="db",
    user="username",
    password="password")

In [3]:
PickingQuery = ('''--Picking____-----------------------																
--Picking Productivity Query																
------------------------------------																
Select 	src.emailaddress															
		,'2020-11-05 00:00:00' as Picking_Date														
		,src.totalnotfound
		,src.B2B_Picking_Qty
		,src.B2C_Picking_Qty																											
		,src.totalprocess as Total_Picking_Qty														
		,tgt.Total_Working_Hrs as Total_Working_Hrs														
		,Round(((src.totalprocess/tgt.Total_Working_Seconds)*60*60)::numeric,2) as Picking_EPH														
		,src.BabyGear_Qty,src.Consumable_Qty
		,'whracks10' as Warehouse
From( 																
	--To get Pick Details															
Select  "emailaddress", sum("quantity") as totalpick, sum("processquantity") as totalprocess																
		,sum("notfoundquantity") as totalnotfound 														
		,sum(case when businesstype = 'B2B' then processquantity else 0 end) as B2B_Picking_Qty														
		,sum(case when businesstype = 'B2C' then processquantity else 0 end) as B2C_Picking_Qty														
		,sum(case when productcatid = '7' then processquantity else 0 end) as BabyGear_Qty														
		,sum(case when productcatid = '999' then processquantity else 0 end) as Consumable_Qty														
From(--For Final GroupBy																
 Select * From(--For Union																
  Select Distinct J1.id,"emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity"																
	             ,J1.productid, productcatid															
  From( --For Join with fc_productdetails																
   SELECT "id","emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity","productid"																
        FROM whracks10.orderitemsrackpickdetails																
        WHERE   (																
        "processquantity" >0																
        OR "notfoundquantity" >0																
         )																
        AND   (																
        "taskid" IN(SELECT distinct "taskid"																
                    FROM whracks10.taskmaster																
                    WHERE "tasktype" = 'Picking'																
                    AND   (																
                    ("startdate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59' or  "enddate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59')																
                    OR ('2020-11-05 00:00:00' between "startdate" and enddate or '2020-11-05 11:59:59' between "startdate" and enddate)																
                 		)														
        			)													
			)													
        AND "lastmodifieddate" < '2020-11-05 11:59:59'																
        AND "lastmodifieddate" > '2020-11-05 00:00:00' 																
	)J1															
	Left Join orderworkflow.fc_productdetails as J2															
	on J1.productid = J2.productid)U															
Union All																
 Select Distinct J1.id,"emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity"																
				,J1.productid, productcatid												
 From( --For Join with fc_productdetails																
	SELECT "id","emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity","productid"															
    FROM whracksarchive10.orderitemsrackpickdetails																
    WHERE ("processquantity" >0 OR "notfoundquantity" >0)																
        AND (																
        	taskid IN(SELECT distinct "taskid"															
                    FROM whracksarchive10.taskmaster																
                    WHERE "tasktype" = 'Picking'																
                    AND   (																
                    ("startdate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59' or  "enddate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59')																
                    OR ('2020-11-05 00:00:00' between "startdate" and enddate or '2020-11-05 11:59:59' between "startdate" and enddate)																
                 			)													
        				)												
		)														
        AND "lastmodifieddate" < '2020-11-05 11:59:59'																
        AND "lastmodifieddate" > '2020-11-05 00:00:00' 																
	)J1															
	Left Join orderworkflow.fc_productdetails as J2															
	on J1.productid = J2.productid															
)Final1																
Group By 1																
Order By 1																
)src																
-- get total working hours: 																
Left Join (																
Select emailaddress																
		,Sum(Case When Working_Seconds < 0 
			 Then Working_interval+'2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp 
			 Else Working_interval End) as Total_Working_Hrs														
		,Sum(Case When Working_Seconds < 0 
			 Then Working_Seconds+(date_part('day','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*24*60*60														
								+ date_part('hour','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*60*60
								+ date_part('minute','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*60
								+ date_part('second','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp ) )  
			 Else Working_Seconds End) As Total_Working_Seconds													
From(																
Select 	emailaddress															
		,Sum(Case When status in ('out') then diff_interval end)
			-Sum(Case When status = 'in' then diff_interval end) as Working_interval														
		,Sum(Case When status in ('out') then diff_in_seconds end)
			-Sum(Case When status = 'in' then diff_in_seconds end) as Working_Seconds														
From(
Select emailaddress, status,  actiontime
	,date_part('day',actiontime-'2020-11-05 00:00:00' )*1440*60 												
					+ date_part('hour',actiontime-'2020-11-05 00:00:00' )*60*60 											
					+ date_part('minute',actiontime-'2020-11-05 00:00:00' )*60
					+ date_part('second',actiontime-'2020-11-05 00:00:00' )as diff_in_seconds											
	,actiontime-'2020-11-05 00:00:00'::timestamp as diff_interval
From (
	SELECT emailaddress, status,  actiontime, logid
  	  ,lag(status) OVER (Partition by emailaddress ORDER BY logid) as prev_to
	FROM whracks10.userworkdetails_log
	Where actiontime between '2020-11-05 00:00:00' and '2020-11-05 11:59:59'
    AND status in ('in', 'out') 
	AND modulename in ('wave_picking', 'orderbasepicking', 'b2c_orderbasepicking', 'b2b_orderbasepicking'
							   , 'mark_bin_close', 'rack_wise_wave_picking')
	Order by emailaddress, logid
) table1
	Where prev_to ISNULL or status<> prev_to
) WH																
Group By 1																
)WH																
Group By 1															
)tgt 																
On src.emailaddress=tgt.emailaddress																
Order By src.emailaddress																''')

In [4]:
ConsoPickingQuery = ('''--ConsoPick___----------------------------------																
--CONSO Picker Productivity Query																
---------------------------------------------																
Select 	src.emailaddress
		,'2020-11-05 00:00:00' as Conso_Picking_Date														
		,src.totalnotfound													
		,src.B2B_Picking_Qty as B2B_Qty														
		,src.B2C_Picking_Qty as B2C_Qty														
		,src.totalprocess as Total_Picking_Qty														
		,tgt.Total_Working_Hrs as Total_Working_Hrs														
		,Round(((src.totalprocess/tgt.Total_Working_Seconds)*60*60)::numeric,2) as EPH														
		,src.BabyGear_Qty,src.Consumable_Qty
		,'whracks10' as Warehouse
From( 																
	--To get Conso Pick Details															
Select  "emailaddress", sum("quantity") as totalpick, sum("processquantity") as totalprocess																
		,sum("notfoundquantity") as totalnotfound 														
		,sum(case when businesstype = 'B2B' then processquantity else 0 end) as B2B_Picking_Qty														
		,sum(case when businesstype = 'B2C' then processquantity else 0 end) as B2C_Picking_Qty														
		,sum(case when productcatid = '7' then processquantity else 0 end) as BabyGear_Qty														
		,sum(case when productcatid = '999' then processquantity else 0 end) as Consumable_Qty														
From(--For Final GroupBy																
 Select * From(--For Union																
  Select Distinct J1.id,"emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity"																
	             ,J1.productid, productcatid															
  From( --For Join with fc_productdetails																
   SELECT "id","emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity","productid"																
        FROM whracks10.orderitemsrackconsodetails																
        WHERE   (																
        "processquantity" >0																
        OR "notfoundquantity" >0																
         )																
        AND   (																
        "taskid" IN(SELECT distinct "taskid"																
                    FROM whracks10.taskmaster																
                    WHERE "tasktype" = 'ConsoOut'																
                    AND   (																
                    ("startdate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59' or  "enddate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59')																
                    OR ('2020-11-05 00:00:00' between "startdate" and enddate or '2020-11-05 11:59:59' between "startdate" and enddate)																
                 		)														
        			)													
			)													
        AND "lastmodifieddate" < '2020-11-05 11:59:59'																
        AND "lastmodifieddate" > '2020-11-05 00:00:00' 																
	)J1															
	Left Join orderworkflow.fc_productdetails as J2															
	on J1.productid = J2.productid)U															
Union All																
 Select Distinct J1.id,"emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity"																
				,J1.productid, productcatid												
 From( --For Join with fc_productdetails																
	SELECT "id","emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity","productid"															
    FROM whracksarchive10.orderitemsrackconsodetails																
    WHERE ("processquantity" >0 OR "notfoundquantity" >0)																
        AND (																
        	taskid IN(SELECT distinct "taskid"															
                    FROM whracksarchive10.taskmaster																
                    WHERE "tasktype" = 'ConsoOut'																
                    AND   (																
                    ("startdate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59' or  "enddate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59')																
                    OR ('2020-11-05 00:00:00' between "startdate" and enddate or '2020-11-05 11:59:59' between "startdate" and enddate)																
                 			)													
        				)												
		)														
        AND "lastmodifieddate" < '2020-11-05 11:59:59'																
        AND "lastmodifieddate" > '2020-11-05 00:00:00' 																
	)J1															
	Left Join orderworkflow.fc_productdetails as J2															
	on J1.productid = J2.productid															
)Final1																
Group By 1																
Order By 1																
)src																
-- get total working hours: 																
Left Join (																
Select emailaddress																
		,Sum(Case When Working_Seconds < 0 
			 Then Working_interval+'2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp 
			 Else Working_interval End) as Total_Working_Hrs														
		,Sum(Case When Working_Seconds < 0 
			 Then Working_Seconds+(date_part('day','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*24*60*60														
								+ date_part('hour','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*60*60
								+ date_part('minute','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*60
								+ date_part('second','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp ) )  
			 Else Working_Seconds End) As Total_Working_Seconds													
From(																
Select 	emailaddress															
		,Sum(Case When status in ('out') then diff_interval end)
			-Sum(Case When status = 'in' then diff_interval end) as Working_interval														
		,Sum(Case When status in ('out') then diff_in_seconds end)
			-Sum(Case When status = 'in' then diff_in_seconds end) as Working_Seconds														
From(
Select emailaddress, status,  actiontime
	,date_part('day',actiontime-'2020-11-05 00:00:00' )*1440*60 												
					+ date_part('hour',actiontime-'2020-11-05 00:00:00' )*60*60 											
					+ date_part('minute',actiontime-'2020-11-05 00:00:00' )*60
					+ date_part('second',actiontime-'2020-11-05 00:00:00' )as diff_in_seconds											
	,actiontime-'2020-11-05 00:00:00'::timestamp as diff_interval
From(
	SELECT emailaddress, status,  actiontime, logid
 	   ,lag(status) OVER (Partition by emailaddress ORDER BY logid) as prev_to
	FROM whracks10.userworkdetails_log
	Where actiontime between '2020-11-05 00:00:00' and '2020-11-05 11:59:59'
    AND status in ('in', 'out') 
	AND modulename in ('b2c_pick_completed_orders', 'b2c_not_found_list')	
	order by emailaddress, logid
) table1
	Where prev_to ISNULL or status<> prev_to												
)WH																
Group By 1																
)WH																
Group By 1																
)tgt 																
On src.emailaddress=tgt.emailaddress																
Order By src.emailaddress																''')

In [5]:
ConsoRackingQuery = ('''--ConsoRack___----------------------------------																
--Conso Rackin Productivity Query
------------------------------------																
Select 	src.emailaddress															
		,'2020-11-05 00:00:00' as Conso_Racking_Date														
		,src.totalnotfound													
		,src.B2B_Picking_Qty as B2B_Qty														
		,src.B2C_Picking_Qty as B2C_Qty														
		,src.totalprocess as Total_Picking_Qty														
		,tgt.Total_Working_Hrs as Total_Working_Hrs														
		,Round(((src.totalprocess/tgt.Total_Working_Seconds)*60*60)::numeric,2) as EPH														
		,src.BabyGear_Qty,src.Consumable_Qty
		,'whracks10' as Warehouse
From( 																
	--To get Conso Rack Details															
Select  "emailaddress", sum("quantity") as totalpick, sum("processquantity") as totalprocess																
		,sum("notfoundquantity") as totalnotfound 														
		,sum(case when businesstype = 'B2B' then processquantity else 0 end) as B2B_Picking_Qty														
		,sum(case when businesstype = 'B2C' then processquantity else 0 end) as B2C_Picking_Qty														
		,sum(case when productcatid = '7' then processquantity else 0 end) as BabyGear_Qty														
		,sum(case when productcatid = '999' then processquantity else 0 end) as Consumable_Qty														
From(--For Final GroupBy																
 Select * From(--For Union																
  Select Distinct J1.id,"emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity"																
	             ,J1.productid, productcatid															
  From( --For Join with fc_productdetails																
   SELECT "id","emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity","productid"																
        FROM whracks10.orderitemsrackpickdetails																
        WHERE   (																
        "processquantity" >0																
        OR "notfoundquantity" >0																
         )																
        AND   (																
        "taskid" IN(SELECT distinct "taskid"																
                    FROM whracks10.taskmaster																
                    WHERE "tasktype" = 'Conso'
					AND taskname in ('B2CExcessRackIn', 'B2BExcessRackIn'
									 , 'B2BConsoRackIn', 'B2CConsoRackIn')
                    AND   (																
                    ("startdate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59' or  "enddate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59')																
                    OR ('2020-11-05 00:00:00' between "startdate" and enddate or '2020-11-05 11:59:59' between "startdate" and enddate)																
                 		)														
        			)													
			)													
        AND "lastmodifieddate" < '2020-11-05 11:59:59'																
        AND "lastmodifieddate" > '2020-11-05 00:00:00' 																
	)J1															
	Left Join orderworkflow.fc_productdetails as J2															
	on J1.productid = J2.productid)U															
Union All																
 Select Distinct J1.id,"emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity"																
				,J1.productid, productcatid												
 From( --For Join with fc_productdetails																
	SELECT "id","emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity","productid"															
    FROM whracksarchive10.orderitemsrackpickdetails																
    WHERE ("processquantity" >0 OR "notfoundquantity" >0)																
        AND (																
        	taskid IN(SELECT distinct "taskid"															
                    FROM whracksarchive10.taskmaster																
                    WHERE "tasktype" = 'Conso'
					AND taskname in ('B2CExcessRackIn', 'B2BExcessRackIn'
									 , 'B2BConsoRackIn', 'B2CConsoRackIn')
                    AND   (																
                    ("startdate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59' or  "enddate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59')																
                    OR ('2020-11-05 00:00:00' between "startdate" and enddate or '2020-11-05 11:59:59' between "startdate" and enddate)																
                 			)													
        				)												
		)														
        AND "lastmodifieddate" < '2020-11-05 11:59:59'																
        AND "lastmodifieddate" > '2020-11-05 00:00:00' 																
	)J1															
	Left Join orderworkflow.fc_productdetails as J2															
	on J1.productid = J2.productid															
)Final1																
Group By 1																
Order By 1																
)src																
-- get total working hours: 																
Left Join (																
Select emailaddress																
		,Sum(Case When Working_Seconds < 0 
			 Then Working_interval+'2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp 
			 Else Working_interval End) as Total_Working_Hrs														
		,Sum(Case When Working_Seconds < 0 
			 Then Working_Seconds+(date_part('day','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*24*60*60														
								+ date_part('hour','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*60*60
								+ date_part('minute','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*60
								+ date_part('second','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp ) )  
			 Else Working_Seconds End) As Total_Working_Seconds													
From(																
Select 	emailaddress															
		,Sum(Case When status in ('out') then diff_interval end)
			-Sum(Case When status = 'in' then diff_interval end) as Working_interval														
		,Sum(Case When status in ('out') then diff_in_seconds end)
			-Sum(Case When status = 'in' then diff_in_seconds end) as Working_Seconds														
From(
Select emailaddress, status,  actiontime
	,date_part('day',actiontime-'2020-11-05 00:00:00' )*1440*60 												
					+ date_part('hour',actiontime-'2020-11-05 00:00:00' )*60*60 											
					+ date_part('minute',actiontime-'2020-11-05 00:00:00' )*60
					+ date_part('second',actiontime-'2020-11-05 00:00:00' )as diff_in_seconds											
	,actiontime-'2020-11-05 00:00:00'::timestamp as diff_interval
From(
	SELECT emailaddress, status,  actiontime, logid
 	   ,lag(status) OVER (Partition by emailaddress ORDER BY logid) as prev_to
	FROM whracks10.userworkdetails_log
	Where actiontime between '2020-11-05 00:00:00' and '2020-11-05 11:59:59'
    AND status in ('in', 'out') 
	AND modulename in ('b2c_consolidate_orders', 'b2b_consolidate_orders', 'b2c_add_excess_items', 
							'b2b_add_excess_items', 'b2b_consolidate_orders_b2b')	
	order by emailaddress, logid
) table1
	Where prev_to ISNULL or status<> prev_to												
)WH																
Group By 1																
)WH																
Group By 1																
)tgt 																
On src.emailaddress=tgt.emailaddress																
Order By src.emailaddress																''')

In [6]:
PackingQuery = ('''--Packing____----------------------------------																
--Packing Productivity Query																
---------------------------------------------																
Select 	src.emailaddress
		,'2020-11-05 00:00:00' as Conso_Picking_Date														
		,src.totalnotfound													
		,src.B2B_Picking_Qty as B2B_Qty														
		,src.B2C_Picking_Qty as B2C_Qty														
		,src.totalprocess as Total_Picking_Qty														
		,tgt.Total_Working_Hrs as Total_Working_Hrs														
		,Round(((src.totalprocess/tgt.Total_Working_Seconds)*60*60)::numeric,2) as EPH														
		,src.BabyGear_Qty,src.Consumable_Qty
		,'whracks10' as Warehouse
From( 																
	--To get Packing Details															
Select  "emailaddress", sum("quantity") as totalpick, sum("processquantity") as totalprocess																
		,sum("notfoundquantity") as totalnotfound 														
		,sum(case when businesstype = 'B2B' then processquantity else 0 end) as B2B_Picking_Qty														
		,sum(case when businesstype = 'B2C' then processquantity else 0 end) as B2C_Picking_Qty														
		,sum(case when productcatid = '7' then processquantity else 0 end) as BabyGear_Qty														
		,sum(case when productcatid = '999' then processquantity else 0 end) as Consumable_Qty														
From(--For Final GroupBy																
 Select * From(--For Union																
  Select Distinct J1.id,"emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity"																
	             ,J1.productid, productcatid															
  From( --For Join with fc_productdetails																
   SELECT "id","emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity","productid"																
        FROM whracks10.packingbinitemdetails																
        WHERE   (																
        "processquantity" >0																
        OR "notfoundquantity" >0																
         )																
        AND   (																
        "taskid" IN(SELECT distinct "taskid"																
                    FROM whracks10.taskmaster																
                    WHERE "tasktype" = 'Packing'																
                    AND   (																
                    ("startdate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59' or  "enddate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59')																
                    OR ('2020-11-05 00:00:00' between "startdate" and enddate or '2020-11-05 11:59:59' between "startdate" and enddate)																
                 		)														
        			)													
			)													
        AND "lastmodifieddate" < '2020-11-05 11:59:59'																
        AND "lastmodifieddate" > '2020-11-05 00:00:00' 																
	)J1															
	Left Join orderworkflow.fc_productdetails as J2															
	on J1.productid = J2.productid)U															
Union All																
 Select Distinct J1.id,"emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity"																
				,J1.productid, productcatid												
 From( --For Join with fc_productdetails																
	SELECT "id","emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity","productid"															
    FROM whracksarchive10.packingbinitemdetails																
    WHERE ("processquantity" >0 OR "notfoundquantity" >0)																
        AND (																
        	taskid IN(SELECT distinct "taskid"															
                    FROM whracksarchive10.taskmaster																
                    WHERE "tasktype" = 'Packing'																
                    AND   (																
                    ("startdate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59' or  "enddate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59')																
                    OR ('2020-11-05 00:00:00' between "startdate" and enddate or '2020-11-05 11:59:59' between "startdate" and enddate)																
                 			)													
        				)												
		)														
        AND "lastmodifieddate" < '2020-11-05 11:59:59'																
        AND "lastmodifieddate" > '2020-11-05 00:00:00' 																
	)J1															
	Left Join orderworkflow.fc_productdetails as J2															
	on J1.productid = J2.productid															
)Final1																
Group By 1																
Order By 1																
)src																
-- get total working hours: 																
Left Join (																
Select emailaddress																
		,Sum(Case When Working_Seconds < 0 
			 Then Working_interval+'2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp 
			 Else Working_interval End) as Total_Working_Hrs														
		,Sum(Case When Working_Seconds < 0 
			 Then Working_Seconds+(date_part('day','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*24*60*60														
								+ date_part('hour','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*60*60
								+ date_part('minute','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*60
								+ date_part('second','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp ) )  
			 Else Working_Seconds End) As Total_Working_Seconds													
From(																
Select 	emailaddress															
		,Sum(Case When status in ('out') then diff_interval end)
			-Sum(Case When status = 'in' then diff_interval end) as Working_interval														
		,Sum(Case When status in ('out') then diff_in_seconds end)
			-Sum(Case When status = 'in' then diff_in_seconds end) as Working_Seconds														
From(
Select emailaddress, status,  actiontime
	,date_part('day',actiontime-'2020-11-05 00:00:00' )*1440*60 												
					+ date_part('hour',actiontime-'2020-11-05 00:00:00' )*60*60 											
					+ date_part('minute',actiontime-'2020-11-05 00:00:00' )*60
					+ date_part('second',actiontime-'2020-11-05 00:00:00' )as diff_in_seconds											
	,actiontime-'2020-11-05 00:00:00'::timestamp as diff_interval
From(
	SELECT emailaddress, status,  actiontime, logid
 	   ,lag(status) OVER (Partition by emailaddress ORDER BY logid) as prev_to
	FROM whracks10.userworkdetails_log
	Where actiontime between '2020-11-05 00:00:00' and '2020-11-05 11:59:59'
    AND status in ('in', 'out') 
	AND modulename in ('bin/box_packing','b2b_packing')	
	order by emailaddress, logid
) table1
	Where prev_to ISNULL or status<> prev_to												
)WH																
Group By 1																
)WH																
Group By 1																
)tgt 																
On src.emailaddress=tgt.emailaddress																
Order By src.emailaddress																''')

In [7]:
QCQuery = ('''--QC_________----------------------------------																
--QC Productivity Query																
---------------------------------------------																
Select 	src.emailaddress
		,'2020-11-05 00:00:00' as QC_Date														
		,src.totalnotfound													
		,src.B2B_Picking_Qty as B2B_Qty														
		,src.B2C_Picking_Qty as B2C_Qty														
		,src.totalprocess as Total_Picking_Qty														
		,tgt.Total_Working_Hrs as Total_Working_Hrs														
		,Round(((src.totalprocess/tgt.Total_Working_Seconds)*60*60)::numeric,2) as EPH														
		,src.BabyGear_Qty,src.Consumable_Qty
		,'whracks10' as Warehouse
From( 																
	--To get QC Details															
Select  "emailaddress", sum("quantity") as totalpick, sum("processquantity") as totalprocess																
		,sum("notfoundquantity") as totalnotfound 														
		,sum(case when businesstype = 'B2B' then processquantity else 0 end) as B2B_Picking_Qty														
		,sum(case when businesstype = 'B2C' then processquantity else 0 end) as B2C_Picking_Qty														
		,sum(case when productcatid = '7' then processquantity else 0 end) as BabyGear_Qty														
		,sum(case when productcatid = '999' then processquantity else 0 end) as Consumable_Qty														
From(--For Final GroupBy																
 Select * From(--For Union																
  Select Distinct J1.id,"emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity"																
	             ,J1.productid, productcatid															
  From( --For Join with fc_productdetails																
   SELECT "id","emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity","productid"																
        FROM whracks10.orderitemsrackqcdetails																
        WHERE   (																
        "processquantity" >0																
        OR "notfoundquantity" >0																
         )																
        AND   (																
        "taskid" IN(SELECT distinct "taskid"																
                    FROM whracks10.taskmaster																
                    WHERE "tasktype" = 'QCShipping'																
                    AND   (																
                    ("startdate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59' or  "enddate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59')																
                    OR ('2020-11-05 00:00:00' between "startdate" and enddate or '2020-11-05 11:59:59' between "startdate" and enddate)																
                 		)														
        			)													
			)													
        AND "lastmodifieddate" < '2020-11-05 11:59:59'																
        AND "lastmodifieddate" > '2020-11-05 00:00:00' 																
	)J1															
	Left Join orderworkflow.fc_productdetails as J2															
	on J1.productid = J2.productid)U															
Union All																
 Select Distinct J1.id,"emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity"																
				,J1.productid, productcatid												
 From( --For Join with fc_productdetails																
	SELECT "id","emailaddress","businesstype", "quantity", "processquantity", "notfoundquantity","productid"															
    FROM whracksarchive10.orderitemsrackqcdetails																
    WHERE ("processquantity" >0 OR "notfoundquantity" >0)																
        AND (																
        	taskid IN(SELECT distinct "taskid"															
                    FROM whracksarchive10.taskmaster																
                    WHERE "tasktype" = 'QCShipping'																
                    AND   (																
                    ("startdate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59' or  "enddate" between '2020-11-05 00:00:00' and '2020-11-05 11:59:59')																
                    OR ('2020-11-05 00:00:00' between "startdate" and enddate or '2020-11-05 11:59:59' between "startdate" and enddate)																
                 			)													
        				)												
		)														
        AND "lastmodifieddate" < '2020-11-05 11:59:59'																
        AND "lastmodifieddate" > '2020-11-05 00:00:00' 																
	)J1															
	Left Join orderworkflow.fc_productdetails as J2															
	on J1.productid = J2.productid															
)Final1																
Group By 1																
Order By 1																
)src																
-- get total working hours: 																
Left Join (																
Select emailaddress																
		,Sum(Case When Working_Seconds < 0 
			 Then Working_interval+'2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp 
			 Else Working_interval End) as Total_Working_Hrs														
		,Sum(Case When Working_Seconds < 0 
			 Then Working_Seconds+(date_part('day','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*24*60*60														
								+ date_part('hour','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*60*60
								+ date_part('minute','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp )*60
								+ date_part('second','2020-11-05 11:59:59'::timestamp-'2020-11-05 00:00:00'::timestamp ) )  
			 Else Working_Seconds End) As Total_Working_Seconds													
From(																
Select 	emailaddress															
		,Sum(Case When status in ('out') then diff_interval end)
			-Sum(Case When status = 'in' then diff_interval end) as Working_interval														
		,Sum(Case When status in ('out') then diff_in_seconds end)
			-Sum(Case When status = 'in' then diff_in_seconds end) as Working_Seconds														
From(
Select emailaddress, status,  actiontime
	,date_part('day',actiontime-'2020-11-05 00:00:00' )*1440*60 												
					+ date_part('hour',actiontime-'2020-11-05 00:00:00' )*60*60 											
					+ date_part('minute',actiontime-'2020-11-05 00:00:00' )*60
					+ date_part('second',actiontime-'2020-11-05 00:00:00' )as diff_in_seconds											
	,actiontime-'2020-11-05 00:00:00'::timestamp as diff_interval
From(
	SELECT emailaddress, status,  actiontime, logid
 	   ,lag(status) OVER (Partition by emailaddress ORDER BY logid) as prev_to
	FROM whracks10.userworkdetails_log
	Where actiontime between '2020-11-05 00:00:00' and '2020-11-05 11:59:59'
    AND status in ('in', 'out') 
	AND modulename in ('qc_shipping', 'qc-shipping')	
	order by emailaddress, logid
) table1
	Where prev_to ISNULL or status<> prev_to												
)WH																
Group By 1																
)WH																
Group By 1																
)tgt 																
On src.emailaddress=tgt.emailaddress																
Order By src.emailaddress																''')

In [13]:
def productivity_report(query,warehouse_id, start_date, end_date):
    whracks='whracks'+warehouse_id
    whracksarchive='whracksarchive'+warehouse_id
    result=query.replace('2020-11-05 00:00:00',start_date)
    result=result.replace('2020-11-05 11:59:59',end_date)
    result=result.replace('whracks10',whracks)
    result=result.replace('whracksarchive10',whracksarchive)
    result=pd.read_sql(result,conn)
    filename=time.strftime("%Y%m%d%H%M%S")+'WH'+warehouse_id+'_'+query[2:11]+'_'+start_date[0:10]
    result.to_csv('/home/fc/Documents/FC Project/202010 Performance Report/Reports/{}.csv'.format(filename),
                  index=False)
    return result;


In [11]:
df_pick=productivity_report(PickingQuery,'10','2020-11-05 00:00:00','2020-11-05 11:59:59')
df_pick.head(10)

,emailaddress,picking_date,totalnotfound,b2b_picking_qty,b2c_picking_qty,total_picking_qty,total_working_hrs,picking_eph,babygear_qty,consumable_qty,warehouse
0,105539@firstcry.com,2020-11-05 00:00:00,0,170,0,170,03:50:05,44.33,45,0,whracks10
1,105563@firstcry.com,2020-11-05 00:00:00,2,164,59,223,03:06:26,71.77,28,0,whracks10
2,106633@firstcry.com,2020-11-05 00:00:00,0,0,304,304,03:01:16,100.63,0,0,whracks10
3,106646@firstcry.com,2020-11-05 00:00:00,3,0,314,314,03:31:28,89.09,0,0,whracks10
4,106664@firstcry.com,2020-11-05 00:00:00,1,0,279,279,03:40:25,75.95,0,0,whracks10
5,106665@firstcry.com,2020-11-05 00:00:00,0,5,304,309,04:15:05,72.68,0,0,whracks10
6,107202@firstcry.com,2020-11-05 00:00:00,6,292,102,394,03:24:48,115.43,0,4,whracks10
7,107247@firstcry.com,2020-11-05 00:00:00,2,59,133,192,03:12:50,59.74,2,0,whracks10
8,107248@firstcry.com,2020-11-05 00:00:00,0,147,139,286,02:57:00,96.95,2,0,whracks10
9,107253@firstcry.com,2020-11-05 00:00:00,0,319,0,319,01:20:10,238.75,0,0,whracks10


In [12]:
df_ConsoRack=productivity_report(ConsoRackingQuery,'10','2020-11-05 00:00:00','2020-11-05 11:59:59')
df_ConsoRack.head(10)

,emailaddress,conso_racking_date,totalnotfound,b2b_qty,b2c_qty,total_picking_qty,total_working_hrs,eph,babygear_qty,consumable_qty,warehouse
0,103431@firstcry.com,2020-11-05 00:00:00,0,0,291,291,03:28:07,83.90,0,0,whracks10
1,106527@firstcry.com,2020-11-05 00:00:00,0,0,613,613,05:07:00,119.80,0,0,whracks10
2,106554@firstcry.com,2020-11-05 00:00:00,0,75,228,303,04:10:15,72.65,0,1,whracks10
3,106557@firstcry.com,2020-11-05 00:00:00,0,0,607,607,06:09:22,98.60,0,0,whracks10
4,106611@firstcry.com,2020-11-05 00:00:00,0,388,82,470,03:20:49,140.43,0,6,whracks10
5,106612@firstcry.com,2020-11-05 00:00:00,0,0,262,262,03:19:28,78.81,0,0,whracks10
6,106613@firstcry.com,2020-11-05 00:00:00,0,152,70,222,02:36:00,85.38,0,2,whracks10
7,106670@firstcry.com,2020-11-05 00:00:00,1,256,80,336,03:53:20,86.40,0,0,whracks10
8,107403@firstcry.com,2020-11-05 00:00:00,1,0,254,254,03:19:52,76.25,0,0,whracks10
9,107405@firstcry.com,2020-11-05 00:00:00,0,0,293,293,03:01:23,96.92,0,0,whracks10


In [14]:
df_ConsoPick=productivity_report(ConsoPickingQuery,'10','2020-11-05 00:00:00','2020-11-05 11:59:59')
df_ConsoPick.head(10)

,emailaddress,conso_picking_date,totalnotfound,b2b_qty,b2c_qty,total_picking_qty,total_working_hrs,eph,babygear_qty,consumable_qty,warehouse
0,106543@firstcry.com,2020-11-05 00:00:00,0,0,2989,2989,06:22:38,468.70,0,0,whracks10
1,12104@firstcry.com,2020-11-05 00:00:00,0,0,991,991,02:03:35,481.13,0,0,whracks10
2,12410@firstcry.com,2020-11-05 00:00:00,0,0,2975,2975,05:54:50,503.05,0,0,whracks10
3,12578@firstcry.com,2020-11-05 00:00:00,0,0,814,814,01:21:37,598.41,0,0,whracks10
4,180142@firstcry.com,2020-11-05 00:00:00,0,0,866,866,01:52:59,459.89,0,0,whracks10
5,180264@firstcry.com,2020-11-05 00:00:00,0,0,1182,1182,03:16:26,361.04,0,0,whracks10
6,180364@firstcry.com,2020-11-05 00:00:00,0,0,1255,1255,02:37:49,477.14,0,0,whracks10
7,2626@firstcry.com,2020-11-05 00:00:00,0,0,1856,1856,03:58:14,467.44,0,0,whracks10
8,2762@firstcry.com,2020-11-05 00:00:00,0,0,2180,2180,03:44:46,581.94,0,0,whracks10
9,91131@firstcry.com,2020-11-05 00:00:00,0,0,2006,2006,04:46:06,420.69,0,0,whracks10


In [17]:
df_Packing=productivity_report(PackingQuery,'10','2020-11-05 00:00:00','2020-11-05 11:59:59')
df_Packing.head(10)

,emailaddress,conso_picking_date,totalnotfound,b2b_qty,b2c_qty,total_picking_qty,total_working_hrs,eph,babygear_qty,consumable_qty,warehouse
0,105576@firstcry.com,2020-11-05 00:00:00,0,242,0,242,03:36:52,66.95,4,0,whracks10
1,109109@firstcry.com,2020-11-05 00:00:00,0,875,0,875,05:06:50,171.10,36,2,whracks10
2,110785@firstcry.com,2020-11-05 00:00:00,0,8,0,8,01:21:28,5.89,8,0,whracks10
3,111185@firstcry.com,2020-11-05 00:00:00,0,2591,0,2591,03:54:02,664.26,3,1603,whracks10
4,11522@firstcry.com,2020-11-05 00:00:00,0,1131,0,1131,05:19:00,212.73,7,0,whracks10
5,12420@firstcry.com,2020-11-05 00:00:00,0,250,0,250,03:54:42,63.91,12,0,whracks10
6,12422@firstcry.com,2020-11-05 00:00:00,0,987,0,987,03:13:51,305.49,10,22,whracks10
7,12500@firstcry.com,2020-11-05 00:00:00,0,167,0,167,00:52:10,192.08,0,0,whracks10
8,12580@firstcry.com,2020-11-05 00:00:00,0,44,0,44,04:24:18,9.99,1,0,whracks10
9,12602@firstcry.com,2020-11-05 00:00:00,0,602,0,602,03:54:40,153.92,3,10,whracks10


In [19]:
df_QC=productivity_report(QCQuery,'10','2020-11-05 00:00:00','2020-11-05 11:59:59')
df_QC.head(10)

,emailaddress,qc_date,totalnotfound,b2b_qty,b2c_qty,total_picking_qty,total_working_hrs,eph,babygear_qty,consumable_qty,warehouse
0,105579@firstcry.com,2020-11-05 00:00:00,0,0,380,380,03:43:15,102.13,1,0,whracks10
1,106590@firstcry.com,2020-11-05 00:00:00,0,0,52,52,04:10:11,12.47,30,0,whracks10
2,106592@firstcry.com,2020-11-05 00:00:00,0,0,50,50,04:16:36,11.69,16,0,whracks10
3,107177@firstcry.com,2020-11-05 00:00:00,0,0,289,289,03:06:42,92.88,2,0,whracks10
4,107420@firstcry.com,2020-11-05 00:00:00,0,0,345,345,03:45:01,91.99,0,0,whracks10
5,107421@firstcry.com,2020-11-05 00:00:00,0,0,222,222,03:42:40,59.82,2,0,whracks10
6,107422@firstcry.com,2020-11-05 00:00:00,0,0,333,333,03:28:50,95.67,0,0,whracks10
7,107423@firstcry.com,2020-11-05 00:00:00,0,0,360,360,03:58:08,90.71,1,0,whracks10
8,107424@firstcry.com,2020-11-05 00:00:00,0,0,304,304,03:48:11,79.94,1,0,whracks10
9,107434@firstcry.com,2020-11-05 00:00:00,0,0,174,174,03:13:06,54.07,1,0,whracks10
